In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd

# Import your custom CRUD class
from crud2 import CRUD

###########################
# MongoDB Connection Setup
###########################
username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 31937
db_name = "AAC"
collection_name = "animals"

# Instantiate CRUD object
shelter = CRUD(username, password, host, port, db_name, collection_name)

# Pull data from MongoDB
df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'], inplace=True, errors='ignore')  # Remove MongoDB ID column

###########################
# Dashboard Layout
###########################
app = JupyterDash(__name__)

# Load logo image
image_filename = 'GraziosoSalvareLogo.png'  # Make sure this image is in the same folder
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    html.Center([
        html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height': '100px'}),
        html.B(html.H1('Grazioso Salvare Dashboard - Alexander Feeney'))
    ]),
    html.Hr(),

    # Filter Section
    html.Div([
        html.Label("Select Rescue Type"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'block'}
        )
    ]),

    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="native",
        sort_action="native",
        page_action="native",
        page_current=0,
        page_size=10,
        row_selectable="single",
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        style_header={'backgroundColor': 'rgb(210, 210, 210)', 'fontWeight': 'bold'}
    ),

    html.Br(),
    html.Hr(),

    # Graph + Map Section
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Callbacks
#############################################

# Update DataTable based on filter
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    query = {}
    if filter_type == 'Water Rescue':
        query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                 "age_upon_outcome_in_weeks": {"$lte": 156}}
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                                   "Siberian Husky", "Rottweiler"]},
                 "age_upon_outcome_in_weeks": {"$lte": 156}}
    elif filter_type == 'Disaster or Individual Tracking':
        query = {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                                   "Bloodhound", "Rottweiler"]},
                 "age_upon_outcome_in_weeks": {"$lte": 156}}

    result = pd.DataFrame.from_records(shelter.read(query))
    result.drop(columns=['_id'], inplace=True, errors='ignore')
    return result.to_dict('records')

# Pie Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame(viewData)
    if dff.empty:
        return [html.P("No data available")]
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return [dcc.Graph(figure=fig)]

# Highlight selected column
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Map Callback
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData:
        return [html.P("No data to display")]

    dff = pd.DataFrame(viewData)
    row = index[0] if index else 0

    try:
        lat = dff.iloc[row]['location_lat']
        lon = dff.iloc[row]['location_long']
        breed = dff.iloc[row]['breed']
        name = dff.iloc[row]['name']
    except Exception:
        return [html.P("Invalid data for map marker")]

    return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[lat, lon], children=[
            dl.Tooltip(breed),
            dl.Popup([
                html.H1("Animal Name"),
                html.P(name)
            ])
        ])
    ])]

# Run inside notebook
app.run_server(mode="inline", debug=True, port=8050)